In [1]:
import sys
!{sys.executable} -m pip install gensim

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 24.2 MB 245 kB/s eta 0:00:01
     |████████████████████████████████| 92 kB 206 kB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 241 kB/s eta 0:00:01
     |████████████████████████████████| 129 kB 477 kB/s eta 0:00:01
  Using cached bz2file-0.98.tar.gz (11 kB)
     |████████████████████████████████| 69 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 140 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.10.1-py2-none-any.whl size=76878 sha256=ebef308fd34d46460705cf051285cd6bb71c431ef0e35ed340b31cb94e5ec84e
  S

In [8]:
!{sys.executable} -m pip install sklearn

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=497f4a3dee755d87411b72da85aa05a978708eeed6de7e2be197f7ff4e81fb70
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/92/41/c1/46aa0ff43ae6274c41ffea0abc81426980f263d53d30fbc7b9
Successfully built sklearn
You should consider upgrading via the '/usr/local/opt/python@2/bin/python2.7 -m pip install --upgrade pip' command.


In [25]:
!{sys.executable} -m pip install annoy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 646 kB 193 kB/s eta 0:00:01
  Created wheel for annoy: filename=annoy-1.17.0-cp27-cp27m-macosx_10_15_x86_64.whl size=69961 sha256=08242e1fee039dbfcbd858ddfe44f02b48a12d7cb10419d80adbb58b2342e556
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/8a/52/5d/2c304e7e1ff269b7a5688a602e7088af625eda7ea1eb2d4922
Successfully built annoy
You should consider upgrading via the '/usr/local/opt/python@2/bin/python2.7 -m pip install --upgrade pip' command.


In [7]:
# import all dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [9]:
from sklearn import datasets

In [20]:
categories = ["talk.politics.guns", "sci.med"]
cat_dict = {} # Contains raw training data organized by category
for cat in categories:
    cat_dict[cat] = datasets.fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=[cat]).data

In [22]:
import gensim

def tokenize(text, stopwords, max_len = 20):
    return [token for token in gensim.utils.simple_preprocess(text, max_len=max_len) if token not in stopwords]

cat_dict_tagged_train = {} # Contains clean tagged training data organized by category. To be used for the training corpus.
cat_dict_test_clean = {} # Contains clean un-tagged training data organized by category.

offset = 0 # Used for managing IDs of tagged documents
for k, v in cat_dict.items():
    cat_dict_tagged_train[k] = [gensim.models.doc2vec.TaggedDocument(tokenize(text, [], max_len=200), [i+offset]) for i, text in enumerate(v)]
    offset += len(v)
    
# Eventually contains final versions of the training data to actually train the model
train_corpus = [taggeddoc for taggeddoc_list in list(cat_dict_tagged_train.values()) for taggeddoc in taggeddoc_list]

In [23]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=2, epochs=40, window=2)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [27]:
from gensim.similarities.index import AnnoyIndexer

In [28]:
# 100 trees are being used in this example
annoy_index = AnnoyIndexer(model, 100)

/usr/local/lib/python2.7/site-packages/gensim/similarities/index.py:198: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


In [29]:
vector = model.infer_vector(['gun', 'violence'])

In [30]:
print(vector)

[ 0.41937435  0.17563553 -0.12256587  0.10460788 -0.02565083  0.09429242
 -0.02002553 -0.05139732 -0.1040236   0.14222175 -0.25814718 -0.17025264
 -0.10804966 -0.34655386 -0.10933165  0.12939467 -0.28263333 -0.20853165
 -0.24687481  0.03203865  0.15677051  0.03355862  0.40046856  0.0894082
  0.09354968  0.09165044 -0.28075868 -0.36651352 -0.16702029 -0.26822072]


In [34]:
# The instance of AnnoyIndexer we just created is passed
approximate_neighbors = model.wv.most_similar([vector], topn=11, indexer=annoy_index)

# Neatly print the approximate_neighbors and their corresponding cosine similarity values
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

Approximate Neighbors
(761, 0.7355824410915375)
(460, 0.7161076068878174)
(747, 0.71528160572052)
(615, 0.7072787880897522)
(781, 0.7049023509025574)
(423, 0.7028268873691559)
(601, 0.7003947496414185)
(1090, 0.6945617198944092)
(818, 0.6910263299942017)
(506, 0.6896374821662903)
(299, 0.6889603734016418)


In [47]:
model.docvecs[761]

array([ 0.8675403 ,  0.38218212, -0.21814214, -0.30081663,  0.06806248,
        0.01665521, -0.29620945, -0.11488877, -0.7557499 ,  0.19308962,
       -0.5282435 , -0.17090972, -0.38533473, -0.9122857 , -0.24606001,
       -0.11321735, -0.09216058, -0.37829253, -0.4843047 ,  0.12021238,
        0.3243565 ,  0.28818458,  0.9300991 ,  0.41290134,  0.4391494 ,
       -0.2293036 , -0.6845831 , -0.6745128 , -0.40212426, -0.26293546],
      dtype=float32)

In [65]:
# cat_dict_tagged_train['talk.politics.guns'][0].tags
for elem in cat_dict_tagged_train['talk.politics.guns']:
    if elem.tags[0] == 761:
        print(elem)

TaggedDocument([u'even', u'if', u'they', u'outlawed', u'private', u'posession', u'of', u'firearms', u'there', u'would', u'be', u'no', u'moral', u'force', u'behind', u'that', u'law', u'imagine', u'compliance', u'would', u'be', u'low'], [761])
